In [1]:
!pip install transformers langchain scikit-learn
!pip install langchain-community
!pip install sentence-transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, b

In [2]:
!pip install langchain-huggingface

In [3]:
import os

# Replace 'YOUR_HUGGINGFACE_TOKEN' with the token you generated
os.environ["HUGGINGFACE_TOKEN"] = "token here"

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random



In [5]:
# Initialize embedding model with Instructor-XL for RAG
embedding_model_name = "sentence-transformers/sentence-t5-large"
semantic_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
sentence_model = SentenceTransformer(embedding_model_name, trust_remote_code=True)  # For matching

/tmp/ipykernel_30/2256891880.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  semantic_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [6]:
# Initialize LLaMA model for answer generation
generation_model_name = "meta-llama/Llama-3.1-8B"
gen_tokenizer = AutoTokenizer.from_pretrained(generation_model_name, use_auth_token=os.getenv("HUGGINGFACE_TOKEN"))
gen_model = AutoModelForCausalLM.from_pretrained(generation_model_name, use_auth_token=os.getenv("HUGGINGFACE_TOKEN"))
generation_pipeline = pipeline("text-generation", model=gen_model, tokenizer=gen_tokenizer)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
# Wrap the HuggingFace pipeline using LangChain's HuggingFacePipeline
hf_pipeline = HuggingFacePipeline(pipeline=generation_pipeline)

In [8]:
# Load sentiment and emotion analysis models
emotion_model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_pipeline = pipeline("text-classification", model=emotion_model_name)
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipeline = pipeline("text-classification", model=sentiment_model_name)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
import re
# Employee data as LangChain Document objects
employee_data = [
    Document(page_content="John loves pets, has a golden retriever named Max, and enjoys volunteering at the local animal shelter on weekends. He visited Bali last summer and stayed at the Serenity Resort. During his trip, he took a surfing lesson, tried local Balinese cuisine, and attended a cultural dance event. John works as a software engineer in the finance department and has been with the company since 2018. He attended Stanford University, where he studied Computer Science. John’s hobbies include hiking, reading science fiction novels, and playing the guitar. He has a younger sister, Emma, who lives in Boston. Recently, he went on a business trip to Tokyo and enjoyed visiting the Tokyo Tower and trying authentic sushi."),
    Document(page_content="Sarah is from NY, visited family in Canada recently")
]

# Set up FAISS index for context retrieval
db = FAISS.from_documents(employee_data, semantic_embeddings)

# Initialize retrieval-based question-answering chain
qa_chain = RetrievalQA.from_chain_type(llm=hf_pipeline, retriever=db.as_retriever(), chain_type="stuff")

def extract_helpful_answer(llm_output):
    # Match "Helpful Answer:" and capture everything until the next blank line
    match = re.search(r"Helpful Answer:\s*(.*?)(?:\n\s*\n|$)", llm_output, re.DOTALL)
    if match:
        return match.group(1).strip()
    return llm_output.strip()

# Retrieve context and generate expected answer
def generate_dynamic_query_and_answer():
    # Dynamically generate a query using the LLM (e.g., based on the context)
    # You can customize the prompt to ensure the LLM generates relevant queries
    context = random.choice(employee_data).page_content
    query = f"Generate a on-point accurate question based on the the above context"
    generated_query = qa_chain({"query": query})["result"]

    # Extract the helpful part of the answer
    question = extract_helpful_answer(generated_query)
    print(question)
    # Wait for user input
    user_answer = input("Please answer the question: ")
    question_with_context = f"Answer the question fully and correctly \"{question}\" based on the above context"
    
    # Generate the helpful answer from the context
    expected_answer = qa_chain({"query": question_with_context})["result"]
    
    return question, user_answer, expected_answer

# Function to analyze sentiment and emotion
def analyze_emotion_and_sentiment(response_text):
    emotion = emotion_pipeline(response_text)[0]
    sentiment = sentiment_pipeline(response_text)[0]
    return emotion, sentiment

# Function to calculate risk score with Instructor embeddings
def calculate_risk_score(expected_answer, user_answer, previous_responses):
    expected_embedding = sentence_model.encode([expected_answer])
    user_embedding = sentence_model.encode([user_answer])
    
    # Similarity score between expected answer and user's answer
    similarity_score = cosine_similarity(expected_embedding, user_embedding)[0][0]
    
    # Consistency score based on previous responses
    prev_embeddings = sentence_model.encode(previous_responses)
    avg_similarity = np.mean([cosine_similarity(user_embedding, prev.reshape(1, -1))[0][0] for prev in prev_embeddings])
    behavioral_deviation = 1 - avg_similarity

    # Final risk score calculation
    risk_score = (1 - similarity_score) * 100 # + behavioral_deviation * 50
    return risk_score

# Evaluate response with RAG and LangChain
def evaluate_employee_response(previous_responses):
    # Dynamically generate a query and its expected answer
    generated_query, user_answer, expected_answer = generate_dynamic_query_and_answer()

    # Extract the helpful part of the answer
    helpful_answer = extract_helpful_answer(expected_answer)
    
    # Perform emotion and sentiment analysis
    emotion, sentiment = analyze_emotion_and_sentiment(user_answer)
    
    # Calculate risk score
    risk_score = calculate_risk_score(helpful_answer, user_answer, previous_responses)

    return {
        "generated_query": generated_query,
        "user_answer": user_answer,
        "expected_answer": expected_answer,
        "helpful_answer": helpful_answer,
        "emotion": emotion,
        "sentiment": sentiment,
        "risk_score": risk_score
    }

def identify_new_words(question, user_answer, model, similarity_threshold=0.85):
    # Tokenize the question and user answer
    question_tokens = question.split()
    user_tokens = user_answer.split()
    
    # Get embeddings for question tokens
    question_embeddings = model.encode(question_tokens)
    
    # Identify new words based on embedding similarity
    new_words = []
    for token in user_tokens:
        token_embedding = model.encode([token])[0]
        # Check similarity with all question token embeddings
        max_similarity = max(cosine_similarity([token_embedding], question_embeddings)[0])
        if max_similarity < similarity_threshold:
            new_words.append(token)
    
    return new_words

def filter_new_embeddings(question, user_answer, model, similarity_threshold=0.85):
    # Identify new words based on embedding similarity
    new_words = identify_new_words(question, user_answer, model, similarity_threshold)
    
    # If no new words, return zero vector
    if not new_words:
        return np.zeros((1, model.get_sentence_embedding_dimension()))
    
    # Join new words into a string and get embeddings
    new_words_text = " ".join(new_words)
    new_words_embedding = model.encode([new_words_text])
    return new_words_embedding

def calculate_risk_score_with_focus(question, expected_answer, user_answer, model):
    # Get embeddings for the expected answer and user answer
    expected_embedding = model.encode([expected_answer])
    user_embedding = model.encode([user_answer])

    # Extract embeddings for new words
    new_words_embedding = filter_new_embeddings(question, user_answer, model)

    # Similarity scores
    overall_similarity = cosine_similarity(expected_embedding, user_embedding)[0][0]
    new_words_similarity = cosine_similarity(expected_embedding, new_words_embedding)[0][0]

    # Weighted risk score
    weight_new_words = 0.85  # Higher weight for newly generated words
    weight_overall = 0.15    # Lower weight for overall similarity
    risk_score = ((1 - new_words_similarity) * weight_new_words + (1 - overall_similarity) * weight_overall) * 100

    return risk_score

# Example usage
# user_answer = "John went to Bali for his vacation."  # Employee-provided answer
previous_responses = ["He went on a business trip to Tokyo recently."]

response_data = evaluate_employee_response(previous_responses)

# Output results
print("Generated Question:", response_data["generated_query"])
print("User Answer:", response_data["user_answer"])
print("Helpful Answer:", response_data["helpful_answer"])
print("Emotion Analysis:", response_data["emotion"])
print("Sentiment Analysis:", response_data["sentiment"])
print("Risk Score:", response_data["risk_score"])

/tmp/ipykernel_30/2839733586.py:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  generated_query = qa_chain({"query": query})["result"]
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Can you tell me what John's favorite hobby is?


Please answer the question:  Hiking


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Question: Can you tell me what John's favorite hobby is?
User Answer: Hiking
Helpful Answer: Sure! Based on the information provided, John's favorite hobby is hiking.
Emotion Analysis: {'label': 'neutral', 'score': 0.5048455595970154}
Sentiment Analysis: {'label': 'neutral', 'score': 0.5996996760368347}
Risk Score: 24.19459819793701


In [10]:
# Sample inputs
question = "Where did you visit last summer vacation?"
user_answer = "I visited last summer."

# Identify new words
new_words = identify_new_words(question, user_answer, sentence_model, similarity_threshold=0.8)
print("New Words:", new_words)

# Get embeddings for new words
new_embeddings = filter_new_embeddings(question, user_answer, sentence_model)
print("New Words Embedding:", new_embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

New Words Embedding: [[-1.05204275e-02 -2.85950053e-04 -1.07152369e-02  5.39868092e-03
   4.53122072e-02  1.41008687e-03 -1.51262162e-02  2.10266039e-02
  -5.66155985e-02  1.49428248e-02  5.05465716e-02  1.58233959e-02
   2.33881176e-02  1.71496719e-02 -7.81658813e-02  1.50103541e-02
  -3.30841951e-02 -1.51137784e-02 -2.65727453e-02  3.07333190e-03
   3.42567377e-02 -6.88858563e-03  1.00393128e-02  2.81233173e-02
   8.25755298e-03  2.53682882e-02 -2.42295284e-02  3.26557383e-02
   6.56585544e-02 -4.56345826e-02  9.75189358e-03 -2.78552044e-02
  -1.79557484e-02  2.73690140e-03 -3.42627219e-03  2.40805279e-03
   1.23802871e-02  5.62250167e-02 -1.29593164e-02  4.46096919e-02
   2.05711331e-02  1.62062943e-02  3.68776508e-02  7.32712308e-03
   2.79035252e-02  7.07202777e-03 -1.29118115e-02 -4.34671529e-02
  -3.10424380e-02 -1.87244993e-02 -1.84544194e-02 -7.84272030e-02
  -7.08148554e-02 -2.13152189e-02  2.98529975e-02 -3.92001085e-02
  -3.99356000e-02  3.58554199e-02  7.40584135e-02 -3.69

In [11]:
# Sample inputs
question = "Where did you visit last summer vacation?"
expected_answer = "I visited Bali last summer."
user_answer = "I visited last summer."

# Calculate risk score
risk_score = calculate_risk_score_with_focus(question, expected_answer, user_answer, sentence_model)

print("Risk Score:", risk_score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Risk Score: 29.58428293466568


In [12]:
# Sample inputs
question = "Where did you visit last summer vacation?"
user_answer = "I visited bali last summer."

# Identify new words
new_words = identify_new_words(question, user_answer, sentence_model, similarity_threshold=0.8)
print("New Words:", new_words)

# Get embeddings for new words
new_embeddings = filter_new_embeddings(question, user_answer, sentence_model)
print("New Words Embedding:", new_embeddings)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

New Words Embedding: [[-2.30948329e-02 -3.35204825e-02 -6.58523408e-04  1.60694141e-02
   1.31811723e-02  1.51649909e-02 -5.37111890e-03  1.83051322e-02
  -2.09322516e-02  2.59388741e-02  6.36973232e-02 -4.25470062e-02
   4.10768166e-02  3.79476175e-02 -6.44154325e-02  1.10687157e-02
  -7.85182696e-03  1.67606305e-02 -2.07709130e-02 -3.11552756e-03
   2.38093901e-02 -1.77313276e-02  2.66034473e-02  5.95751405e-02
  -4.24955128e-04  2.98170815e-03 -3.37927900e-02  2.87713930e-02
   7.12213144e-02 -1.85035802e-02  2.77119856e-02 -1.64577160e-02
  -5.81739396e-02 -1.11115193e-02 -4.97768149e-02  1.69478878e-02
   2.63108090e-02  1.93887502e-02  2.59167030e-02  4.25473452e-02
   2.08175499e-02  3.24524567e-02  3.72273922e-02 -1.05127571e-02
   3.12823653e-02 -1.48175582e-02 -2.84917802e-02 -6.50751824e-03
  -3.21464241e-02 -4.19216566e-02 -4.37297747e-02 -4.61578704e-02
  -2.59011202e-02 -1.51861738e-02 -4.35690070e-03 -7.18475599e-03
  -6.69954419e-02  3.49164233e-02  5.17537557e-02 -6.76

In [13]:
# Sample inputs
question = "Where did you visit last summer vacation?"
expected_answer = "I visited Bali last summer."
user_answer = "I visited Indonesia last summer."

# Calculate risk score
risk_score = calculate_risk_score_with_focus(question, expected_answer, user_answer, sentence_model)

print("Risk Score:", risk_score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Risk Score: 19.510598182678223


In [14]:
# Sample inputs
question = "Where did you visit last summer vacation?"
expected_answer = "I visited Bali last summer."
user_answer = "I toured last summer."

# Calculate risk score
risk_score = calculate_risk_score_with_focus(question, expected_answer, user_answer, sentence_model)

print("Risk Score:", risk_score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Risk Score: 30.396890342235565


In [15]:
# Sample inputs
question = "Where did you visit last summer vacation?"
expected_answer = "I visited Bali last summer."
user_answer = "I visited last summer."

# Calculate risk score
risk_score = calculate_risk_score(expected_answer, user_answer, previous_responses)

print("Risk Score:", risk_score)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Risk Score: 10.246902704238892


In [17]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00


In [ ]:
previous_responses = ["He went on a business trip to Tokyo recently."]

response_data = evaluate_employee_response(previous_responses)

# Output results
print("Generated Question:", response_data["generated_query"])
print("User Answer:", response_data["user_answer"])
print("Helpful Answer:", response_data["helpful_answer"])
print("Emotion Analysis:", response_data["emotion"])
print("Sentiment Analysis:", response_data["sentiment"])
print("Risk Score:", response_data["risk_score"])

In [ ]:
def minimax_decision(risk_score, depth, is_maximizing):
    """
    Simulates decision branching using minimax logic.
    
    Args:
        risk_score (float): The current risk score (0 to 1).
        depth (int): Remaining depth for branching decisions.
        is_maximizing (bool): Whether we are maximizing or minimizing.

    Returns:
        (float, str): The best score and action at the current depth.
    """
    if depth == 0 or risk_score <= 0.1 or risk_score >= 0.9:
        # Terminal condition: no further branching or extreme scores
        return risk_score, "End" if risk_score < 0.3 else "Probe"

    if is_maximizing:
        # Maximize probing (High-risk strategy)
        new_score_probe = min(risk_score + 0.1, 1)  # Increase risk score for probing
        score_probe, _ = minimax_decision(new_score_probe, depth - 1, False)
        return score_probe, "Probe"
    else:
        # Minimize friction (Low-risk strategy)
        new_score_simplify = max(risk_score - 0.1, 0)  # Decrease risk score for simplifying
        score_simplify, _ = minimax_decision(new_score_simplify, depth - 1, True)
        return score_simplify, "Simplify"

def evaluate_employee_response_with_minimax(previous_responses, depth=3):
    """
    Evaluates employee responses and uses Minimax for branching decisions.
    
    Args:
        previous_responses (list): List of previous responses for context.
        depth (int): Depth for minimax branching.

    Returns:
        dict: Evaluation results including branching decision.
    """
    # Generate a query, user response, and expected answer
    generated_query, user_answer, expected_answer = generate_dynamic_query_and_answer()

    # Extract the helpful part of the expected answer
    helpful_answer = extract_helpful_answer(expected_answer)
    
    # Perform emotion and sentiment analysis
    emotion, sentiment = analyze_emotion_and_sentiment(user_answer)
    
    # Calculate risk score
    risk_score = calculate_risk_score(helpful_answer, user_answer, previous_responses)

    # Use minimax branching decision
    _, action = minimax_decision(risk_score, depth, is_maximizing=True)

    return {
        "generated_query": generated_query,
        "user_answer": user_answer,
        "expected_answer": expected_answer,
        "helpful_answer": helpful_answer,
        "emotion": emotion,
        "sentiment": sentiment,
        "risk_score": risk_score,
        "action": action
    }

# Example usage
previous_responses = ["He went on a business trip to Tokyo recently."]
response_data = evaluate_employee_response_with_minimax(previous_responses, depth=3)

# Output results
print("Generated Question:", response_data["generated_query"])
print("User Answer:", response_data["user_answer"])
print("Helpful Answer:", response_data["helpful_answer"])
print("Emotion Analysis:", response_data["emotion"])
print("Sentiment Analysis:", response_data["sentiment"])
print("Risk Score:", response_data["risk_score"])
print("Branching Action:", response_data["action"])
